In [1]:
import numpy as np
import pygmo as pg

In [2]:
function = pg.problem(pg.cec2014(prob_id=6, dim=30))

In [3]:
def test_fitness(array1D):
    return function.fitness(array1D)

In [8]:
def gen_rand(n_size=1):
    '''
    This function return a n_size-dimensional random vector.
    '''
    return np.random.random(n_size)

class NSSDE(object):
    
    def __init__(self, n_gens=10000, n_pop=100, n_dim=30, F=0.8, Cr=0.9,
                 bounds=[-100, 100], scheme='rand/p/bin', p=1, global_max= 600., fitness=test_fitness):
        self.n_gens=n_gens
        self.n_pop=n_pop
        self.n_dim=n_dim
        self.F=F
        self.Cr=Cr
        self.bounds=bounds
        self.scheme=scheme
        self.p=p
        self.population=NSSDE.init_population(self, pop_size=self.n_pop,
                                                     dim=self.n_dim, bounds=self.bounds)
        self.fitness=fitness
        self.MaxEF= 10000*self.n_dim
        self.F_evals = 0
        self.global_max = global_max
        
    def get_F(self):
        return self.F
    def get_Cr(self):
        return self.Cr
    def get_p(self):
        return self.p
    def get_fitness(self):
        return self.fitness
    def get_scheme(self):
        return self.scheme
    def get_n_gens(self):
        return self.n_gens
    def get_bounds(self):
        return self.bounds
    def get_population(self):
        return self.population

    def init_population(self, pop_size, dim, bounds):
        '''
        This function initialize the population to be use in DE
        Arguments:
        pop_size - Number of individuals (there is no default value to this yet.).
        dim - dimension of the search space (default is 1).
        bounds - The inferior and superior limits respectively (default is [-100, 100]).
        '''
        return np.random.uniform(low=bounds[0], high=bounds[1], size=(pop_size, dim))

    def keep_bounds(self, pop, bounds):
        '''
        This function keep the population in the seach space
        Arguments:
        pop - Population;
        bounds - The inferior and superior limits respectively
        '''
        pop[pop<bounds[0]] = bounds[0]; pop[pop>bounds[1]] = bounds[1]
        return pop
    
    def evolution(self):
        r_info = {}
        # ============ Evaluate the initial population ============
        pop_fitness = np.zeros(self.population.shape[0])
        for ind in range(self.population.shape[0]):
            pop_fitness[ind] = self.fitness(self.population[ind])
        best_idx = np.argmin(pop_fitness)
        r_info['log'] = []
        r_info['log'].append((self.F_evals, pop_fitness[best_idx], np.mean(pop_fitness),
                              np.std(pop_fitness), np.max(pop_fitness), np.median(pop_fitness),self.F, self.Cr))
        
        while self.F_evals < self.MaxEF: 
            mutant = np.zeros_like(self.population)
            trial_pop = np.copy(self.population)
            trial_fitness = np.zeros(trial_pop.shape[0])
            
            for ind in range(self.population.shape[0]):
                
                # ============ Adapt F and Cr ============
                NF = self.F
                NCr = self.Cr
                if gen_rand() < 0.1:
                    NF = 0.2 +0.2*gen_rand()
                    NCr = 0.8 +0.2*gen_rand()
                # ============ Mutation Step ============
                tmp_pop = np.delete(self.population, ind, axis=0)
                choices = np.random.choice(tmp_pop.shape[0], 1+2*self.p, replace=False)
                diffs = 0
                for idiff in range(1, len(choices), 2):
                    diffs += NF*((tmp_pop[choices[idiff]]-tmp_pop[choices[idiff+1]]))
                mutant[ind] = tmp_pop[choices[0]] + diffs
                # keep the bounds
                mutant = NSSDE.keep_bounds(self, mutant, bounds=self.bounds)
            
                # ============ Crossover Step ============               
                K = np.random.choice(trial_pop.shape[1])
                for jnd in range(trial_pop.shape[1]):
                    if jnd == K or gen_rand()<NCr:
                        trial_pop[ind][jnd] = mutant[ind][jnd]
                # keep the bounds
                trial_pop = NSSDE.keep_bounds(self, trial_pop, bounds=self.bounds)
            
                trial_fitness[ind] = self.fitness(trial_pop[ind])
                self.F_evals += 1
                if self.F_evals > self.MaxEF-1:
                    r_info['Population'] = self.population
                    r_info['Fitness'] = pop_fitness
                    r_info['Champion'] = pop_fitness[best_idx]
                    r_info['Champion Index'] = best_idx
                    r_info['Function Evals'] = self.F_evals
                    return r_info
                    
                # ============ Selection ============
                if trial_fitness[ind] < pop_fitness[ind]:
                    self.population[ind] = trial_pop[ind]
                    pop_fitness[ind] = trial_fitness[ind]
                    self.F = NF
                    self.Cr = NCr
                    if trial_fitness[ind] < pop_fitness[best_idx]:
                        best_idx = ind
                # Save Log
                r_info['log'].append((self.F_evals, pop_fitness[best_idx], np.mean(pop_fitness),
                              np.std(pop_fitness), np.max(pop_fitness), np.median(pop_fitness),self.F, self.Cr))
            # ========== Local Search =============
            a_1 = gen_rand(); a_2 = gen_rand()
            a_3 = 1.0 - a_1 - a_2
            
            k, r1, r2 = np.random.choice(self.population.shape[0], size=3)
            V = np.zeros_like(self.population[k])
            for jdim in range(self.population.shape[1]):
                V[jdim] = a_1*self.population[k][jdim] + a_2*self.population[best_idx][jdim] + a_3*(self.population[r1][jdim] - self.population[r2][jdim])
                V = NSSDE.keep_bounds(self, V, bounds=self.bounds)

            self.F_evals += 1
            F_V = self.fitness(V)
            if F_V < pop_fitness[k]:
                self.population[k] = V
                pop_fitness[k] = F_V
                if F_V < pop_fitness[best_idx]:
                    best_idx = k
            # Save Log
            r_info['log'].append((self.F_evals, pop_fitness[best_idx], np.mean(pop_fitness),
                                 np.std(pop_fitness), np.max(pop_fitness), np.median(pop_fitness),self.F, self.Cr))
            # Check the stop criteria
            if np.abs(pop_fitness[best_idx] - self.global_max)<1e-6:
                print('Stop criteria... ')
                r_info['Population'] = self.population
                r_info['Fitness'] = pop_fitness
                r_info['Champion'] = pop_fitness[best_idx]
                r_info['Champion Index'] = best_idx
                r_info['Function Evals'] = self.F_evals
                return r_info

        r_info['Population'] = self.population
        r_info['Fitness'] = pop_fitness
        r_info['Champion'] = pop_fitness[best_idx]
        r_info['Champion Index'] = best_idx
        r_info['Function Evals'] = self.F_evals
        return r_info
             

In [9]:
DE = NSSDE()

In [10]:
DE

In [11]:
ev = DE.evolution()

In [12]:
ev.keys()

['Champion',
 'Champion Index',
 'Function Evals',
 'Fitness',
 'Population',
 'log']

In [13]:
ev['log'][-1]

(299999,
 606.30499491675857,
 606.30499491676005,
 5.524057366190546e-13,
 606.3049949167613,
 606.30499491676005,
 array([ 0.26827413]),
 array([ 0.83652893]))

In [ ]:
DE.binomial_crossover(tm,Cr=0.2, p=2, F=0.5)

In [ ]:
arra1 = np.array([0.1, .2, .3])
arra2 = np.array([0.02, .1, .6])

onde_ = np.where(arra2<arra1)

final_array = arra1
print(final_array)
final_array[onde_] = arra2[onde_]
print(final_array)

In [ ]:
DE.gen_rand()

In [ ]:
DE.get_Cr(), DE.get_bounds()

In [ ]:
def gen_rand(n_size=1):
    '''
    This function return a n_size-dimensional random vector.
    '''
    return np.random.random(n_size)

def init_population(pop_size, dim=1, bounds=[-100,100]):
    '''
    This function initialize the population to be use in DE
    Arguments:
    pop_size - Number of individuals (there is no default value to this yet.).
    dim - dimension of the search space (default is 1).
    bounds - The inferior and superior limits respectively (default is [-100, 100]).
    '''
    return np.random.uniform(low=bounds[0], high=bounds[1], size=(pop_size, dim))

def keep_bounds(pop, bounds=[-10, 10]):
    '''
    This function keep the population in the seach space
    Arguments:
    pop - Population;
    bounds - The inferior and superior limits respectively
    '''
    pop[pop<bounds[0]] = bounds[0]; pop[pop>bounds[1]] = bounds[1]
    return pop

def rand_p(pop, p=1, F=0.7):
    '''
    This function is the rand/p mutation scheme, this is a generalization of rand/1 mutation scheme
     from the first DE paper (Storn and Price).
    Arguments:
    pop - Population;
    p - Number of diferences to be used;
    F - The F scale factor for the diferences (default is 0.7);
    '''
    
    choices = np.random.choice(pop.shape[0], 1+2*p, replace=False)
    diffs = 0
    for idiff in range(1, len(choices), 2):
        diffs += F*((pop[choices[idiff]]-pop[choices[idiff+1]]))
    return pop[choices[0]] + diffs

def binomial_crossover(pop, Cr=0.5, mutation_type=rand_p, **kwargs):
    '''
    This function make the binomial crossover.
    Arguments:
    pop - Population;
    mutation_type - mutation scheme (default is ran_p);
    **kwargs - This is relative to the mutation scheme ex. rand_p needs p and F.
    '''
    
    K = np.random.choice(pop.shape[1])
    for ind in range(tmp.shape[0]):
        mutant = mutation_type(pop, **kwargs)
        for jnd in range(tmp.shape[1]):
            if jnd == K or gen_rand()<Cr:
                tmp[ind][jnd] = mutant[jnd]
    return pop

In [ ]:
tmp = init_population(10, dim=5, bounds=[-5, 5])
tmp

In [ ]:
binomial_crossover(tmp,Cr=0.2, mutation_type=rand_p, p=2, F=0.5)

In [ ]:
rand_p(pop=tmp, p=1, F=0.2)

In [ ]:
K = np.random.choice(tmp.shape[1])
K

In [ ]:
print(tmp)
for ind in range(tmp.shape[0]):
    mutant = rand_p(tmp, p=1, F=0.5)
    for jnd in range(tmp.shape[1]):
        if jnd == K or gen_rand()<0.6:
            #print(tmp[ind][jnd])
            tmp[ind][jnd] = mutant[jnd]
tmp

In [ ]:
of

In [ ]:
trials[trials==1] = rand_p(tmp, p=2, F=0.5)

In [ ]:
trials

In [ ]:
tmp[choices[0]]

In [ ]:
for odd in range(1, len(choices), 2):
    print(odd)

In [ ]:
d = 0
for idiff in range(1, len(choices), 2):
    d += 0.1*(tmp[choices[idiff]]-tmp[choices[idiff+1]])


In [ ]:
d

In [ ]:
tmp[choices[1],:]-tmp[choices[2],:], 0.1*(tmp[choices[1],:]-tmp[choices[2],:])

In [ ]:
tmp[choices[0],:] + 0.1*(tmp[choices[1],:]-tmp[choices[2],:])

In [ ]:
idxs = np.ma.masked_where(pop < -10., pop)
idxs

In [ ]:
pop[[idxs.tolist]] = -5.

In [ ]:
pop